### Database Normalization

In this post I am going to normalize [Unesco Heritage Sites](https://whc.unesco.org/en/list/) data which is originally in spreadsheet form.

In [6]:
import pandas as pd


In [8]:

unesco = pd.read_csv('whc-sites-2018-small.csv')
# unesco.head()
unesco.shape

(1044, 11)

Obviously, this data as it is,  has many vertical replications. Data replication takes a lot of memory and makes the database hard to maintain and manipulate. 

To address these issues, we need to copy this dataset into a relational database and use relations to eliminate duplication and make the data integrated and consistent.

Let's start with creating a PostgreSQL database. 

In [3]:
CREATE DATABASE unesco

The databse was successfully created and now I need to create a table and copy the content of whc-sites-2018-small.csv file into it.

In [2]:
DROP TABLE unesco_raw;
CREATE TABLE unesco_raw
 (name TEXT, description TEXT, justification TEXT, year INTEGER,
    longitude FLOAT, latitude FLOAT, area_hectares FLOAT,
    category TEXT, category_id INTEGER, state TEXT, state_id INTEGER,
    region TEXT, region_id INTEGER, iso TEXT, iso_id INTEGER);

Good!
I have a table for keeping the data in its original form.
Let's copy the spreadsheet to the table.

`CSV HEADER` is to have Postgres skip the first row which is the header.

In [ ]:
\copy unesco_raw(name,description,justification,year,longitude,latitude,area_hectares,category,state,region,iso) FROM '...\Database Normalization\whc-sites-2018-small.csv' WITH DELIMITER ',' CSV HEADER;


In [ ]:
SELECT COUNT(*) FROM unesco_raw;

 count
-------
  1044
(1 row)

OK!
I have the data in unesco_raw table. Ready to be normalized. Let's Review NF rules.

#### 1NF Rules

 - Each table cell should contain a single value
 - No duplicated rows or columns
 - Each column must have only one value for each row in the table
 - There must be a primary key for identification
 
 Except for the last item which I will address shortly, my database is compliant with the first set of rules.
 So, I'll go ahead with the next rules.
 
#### 2NF Rules

 - Create separate tables for sets of values that apply to multiple records
 - Relate these tables with a foreign key
 
According to this rule, I have to create separate tables for category, region, state, and iso columns as they are applied to multiple records.
 
 Back to SQL!
  

In [ ]:
CREATE TABLE category (
  id SERIAL,
  name VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);

In [ ]:
CREATE TABLE state (
  id SERIAL,
  name VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);

In [ ]:
CREATE TABLE region (
  id SERIAL,
  name VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);

In [ ]:
CREATE TABLE iso (
  id SERIAL,
  name VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);

In [ ]:
unesco=# \dt

           List of relations
 Schema |    Name    | Type  |  Owner
--------+------------+-------+----------
 public | category   | table | postgres
 public | iso        | table | postgres
 public | region     | table | postgres
 public | state      | table | postgres
 public | unesco_raw | table | postgres
(5 rows)

Fantastic!
I have all the necessary tables in my database ready to be populated with data from unesco_raw table.


In [ ]:
INSERT INTO category (name) SELECT DISTINCT category FROM unesco_raw;

SELECT * FROM category;

 id |   name
----+----------
  1 | Mixed
  2 | Natural
  3 | Cultural
(3 rows)

In [ ]:
INSERT INTO state (name) SELECT DISTINCT state FROM unesco_raw;

SELECT * FROM state;

 id  |                         name
-----+------------------------------------------------------
   1 | Indonesia
   2 | Bangladesh
   3 | Jerusalem (Site proposed by Jordan)
   4 | Iran (Islamic Republic of)
   5 | Kiribati
   6 | Cameroon
   7 | Luxembourg
   8 | C te d'Ivoire
   9 | Sweden
  10 | Viet Nam

-- More --
(163 rows)
    
    
SELECT COUNT(*) FROM state;

 count
-------
   163
(1 row)



In [ ]:
INSERT INTO region (name) SELECT DISTINCT region FROM unesco_raw;

SELECT * FROM region;

 id |              name
----+---------------------------------
  1 | Asia and the Pacific
  2 | Arab States
  3 | Africa
  4 | Latin America and the Caribbean
  5 | Europe and North America
(5 rows)

In [ ]:
INSERT INTO iso (name) SELECT DISTINCT iso FROM unesco_raw WHERE iso != ' ';

SELECT * FROM iso;

 id  | name
-----+------
   1 | np
   2 | vu
   3 | fr
   4 | bh
   5 | sk
   6 | pa
   7 | ke
   8 | me
   9 | bz
  10 | nz
  11 | bg
  12 | ru
  13 | mg
  14 | ni
-- More --
(162 rows)

Great! 

Now I have all the necessary tables populated with relevant data. The tables are however isolated from each other, there is no relation yet. I have a UNRELATIONAL database at this stage. :-D

It's time for making relations. To make relations, all I need to do is populate unesco_raw.category_id, unesco_raw.state_id, unesco_raw.region_id and unesco_raw.iso_id columns with ids from category, state, region, and iso tables respectively.

In [ ]:
UPDATE unesco_raw SET category_id = (SELECT category.id FROM category WHERE category.name=unesco_raw.category);

UPDATE unesco_raw SET state_id = (SELECT state.id FROM state WHERE state.name=unesco_raw.state);

UPDATE unesco_raw SET region_id = (SELECT region.id FROM region WHERE region.name=unesco_raw.region);

UPDATE unesco_raw SET iso_id = (SELECT iso.id FROM iso WHERE iso.name=unesco_raw.iso);


I'm done with making relations.

But the unesco_raw table is still full of redundant data and duplicate columns, hence not normalized.

To make everything clean and tidy, I create a new table and populate it with normalized data from the unesco_raw table.

In [ ]:
CREATE TABLE unesco
    (name TEXT, description TEXT, justification TEXT, year INTEGER,
    longitude FLOAT, latitude FLOAT, area_hectares FLOAT,
    category_id INTEGER, state_id INTEGER, region_id INTEGER, iso_id INTEGER);

In [ ]:
unesco=# \dt

           List of relations
 Schema |    Name    | Type  |  Owner
--------+------------+-------+----------
 public | category   | table | postgres
 public | iso        | table | postgres
 public | region     | table | postgres
 public | state      | table | postgres
 public | unesco     | table | postgres
 public | unesco_raw | table | postgres
(6 rows)

In [ ]:
INSERT INTO unesco (name, description, justification,
                    year, longitude, latitude, 
                    area_hectares, category_id, state_id,
                    region_id, iso_id) 
            SELECT name, description, justification,
                    year, longitude, latitude, 
                    area_hectares, category_id, state_id,
                    region_id, iso_id
            FROM unesco_raw;

In [ ]:
SELECT unesco.name, year, category.name, state.name, region.name, iso.name
  FROM unesco
  JOIN category ON unesco.category_id = category.id
  JOIN iso ON unesco.iso_id = iso.id
  JOIN state ON unesco.state_id = state.id
  JOIN region ON unesco.region_id = region.id
  ORDER BY region.name, unesco.name
  LIMIT 3;
    
    
              name               | year |   name   |     name     |  name  | name
---------------------------------+------+----------+--------------+--------+------
  Khomani Cultural Landscape     | 2017 | Cultural | South Africa | Africa | za
 Aapravasi Ghat                  | 2006 | Cultural | Mauritius    | Africa | mu
 Air and T n r  Natural Reserves | 1991 | Natural  | Niger        | Africa | ne
(3 rows)

Now I can delete unesco_raw table as it is no longer needed.

And now 3NF;

##### 3NF

 - Eliminate fields that do not depend on the key.
 
In order to investigate compliance with this rule more precisely, I need to know more about the real-world application of a database. This database is now per se compliant with this rule as all fields in the unesco table are dependent and relevant to the key.

#### Done with Normalizing the database.

:-)